In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.neighbors import radius_neighbors_graph

In [2]:
lpmt_pos = pd.read_csv('./data/phase_1/pmts_pos.csv')
train_hits = pd.read_hdf('./data/phase_1/lpmt_hits.h5', mode='r') 
true_info = pd.read_csv('./data/phase_1/true_info.csv') 

In [3]:
message_size = 8
node_features_size = 6
node_state_size = 16

In [4]:
train = train_hits
#train = train_hits.merge(lpmt_pos, how='left', left_on='pmtID', right_on='pmt_id')
#del train['pmt_id']

In [5]:
true_info.head()

,E,R,evtID,x,y,z
0,4.747791,14610.378,0,8290.7790,11995.6180,911.74286
1,3.919721,14630.141,1,11397.6320,5407.4497,-7409.08200
2,6.823932,14573.132,2,14063.3380,-3812.8540,246.65280
3,3.765940,16820.080,3,-2377.9307,-16317.7020,3315.59030
4,3.217473,13026.938,4,-8617.1170,868.5116,9730.98600


In [6]:
train.head() 

,event,hitTime,pmtID
0,0,249.992615,14175
1,0,40.010311,17319
2,0,162.123199,16882
3,0,51.875614,14951
4,0,79.817497,10947


In [7]:
hitTime_mean = np.mean(train['hitTime']) 
hitTime_std = np.std(train['hitTime'])
train['hitTime'] = (train['hitTime'] - hitTime_mean) / hitTime_std

In [8]:
lpmt_pos = lpmt_pos[lpmt_pos['pmt_id'] < 300000]

In [9]:
pos = lpmt_pos[["pmt_x", "pmt_y", "pmt_z"]].values

In [10]:
pos.shape[0]

17739

In [11]:
adjacency_matrix = radius_neighbors_graph(X=pos, 
                radius=1000, mode='connectivity')


In [12]:
from numpy.linalg import norm
from numpy import dot

In [13]:
def cos(a, b):
     return dot(a,b) / norm(a) / norm(b)

In [14]:
def zero_padding(l):
    return np.array(list(itertools.zip_longest(*l, fillvalue=0))).T

In [15]:
E_mean = np.mean(true_info.E)
E_std = np.std(true_info.E)

In [16]:
R_mean = np.mean(true_info.R)
R_std = np.std(true_info.R)

In [17]:
def build_graph():
    in_pmt_ids_val = []
    edges_features_val = []
    out_pmt_ids_val = []
    mask_val = []

    for row in range(pos.shape[0]):
        rows, cols = np.where(adjacency_matrix[row].toarray() > 0)
    
        row_out_pmt = []
        row_mask = []
        
        for col in cols:
            x, y, z = pos[row] - pos[col]
            c = ((cos(pos[row], pos[col]) - 0.997) / 0.002)
                
            in_pmt_ids_val.append(row)                
            edges_features_val.append([x, y, z, c])            
            row_out_pmt.append(col)
            row_mask.append(1)
    
        out_pmt_ids_val.append(row_out_pmt)
        mask_val.append(row_mask)    
    
    return (in_pmt_ids_val,
            edges_features_val,    
            zero_padding(out_pmt_ids_val),
            zero_padding(mask_val))

In [18]:
in_pmt_ids_val, edge_features_val, out_pmt_ids_val, mask_val = build_graph()

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
device

device(type='cuda')

In [22]:
len(train.event.unique())

16000

In [23]:
data_map = {}

for event_id in tqdm(train.event.unique()):    
    event = train[train['event'] == event_id].copy()
    data = lpmt_pos.copy()    
    event['pmtID'] = event['pmtID'].map({v : i for i, v in enumerate(lpmt_pos['pmt_id'])})
    data['pmt_id'] = data['pmt_id'].map({v : i for i, v in enumerate(lpmt_pos['pmt_id'])})
    
    data["hit_count"] = data.pmt_id.map(event.groupby('pmtID').hitTime.count()).fillna(0)
    data["mean_hit_time"] = data.pmt_id.map(event.groupby('pmtID').hitTime.mean()).fillna(0)
    data["min_hit_time"] = data.pmt_id.map(event.groupby('pmtID').hitTime.min()).fillna(0)
    data['pmt_x'] /= 10000.0
    data['pmt_y'] /= 10000.0
    data['pmt_z'] /= 10000.0
    
    X_val = data[['hit_count', 'mean_hit_time', 'min_hit_time', 'pmt_x', 'pmt_y', 'pmt_z']].values 
    E_val = true_info[true_info.evtID == event_id].E.values 
    R_val = true_info[true_info.evtID == event_id].R.values     
    data_map[event_id] = (X_val, (E_val - E_mean) / E_std, (R_val - R_mean) / R_std)

In [24]:
class InitialNet(nn.Module):
    def __init__(self):
        super(InitialNet, self).__init__()        
        self.fc1 = nn.Linear(node_features_size, 32)
        self.fc2 = nn.Linear(32, node_state_size)            


    def forward(self, x):        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)        
        return x
    
class MessageNet(nn.Module):
    def __init__(self):
        super(MessageNet, self).__init__()        
        self.fc1 = nn.Linear(node_state_size, 32)
        self.fc2 = nn.Linear(4, 32)
        self.fc3 = nn.Linear(32, message_size)            


    def forward(self, prev_state, edge_features):        
        x = F.relu(self.fc1(prev_state) + self.fc2(edge_features))
        x = self.fc3(x)        
        return x
    

class UpdateNet(nn.Module):
    def __init__(self):
        super(UpdateNet, self).__init__()        
        self.fc1 = nn.Linear(message_size, 32)
        self.fc2 = nn.Linear(node_features_size, 32)
        self.fc3 = nn.Linear(32, node_state_size)            


    def forward(self, measage, node_features):        
        x = F.relu(self.fc1(measage) + self.fc2(node_features))
        x = self.fc3(x)        
        return x
    
class PredictNet(nn.Module):
    def __init__(self):
        super(PredictNet, self).__init__()        
        self.fc1 = nn.Linear(node_state_size, 32)
        self.fc2 = nn.Linear(32, 1)


    def forward(self, x):        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)        
        return x

class MainNet(nn.Module):
    def __init__(self):
        super(MainNet, self).__init__()      
        self.initial_net = InitialNet()
        self.message_net = MessageNet()
        self.update_net = UpdateNet()
        self.predict_net = PredictNet().to(device)
        

In [25]:
def update_state(state):
    in_state = torch.index_select(state, 0, in_hit_ids)
    messages = main_net.message_net(in_state, edge_features)
    index = torch.flatten(out_hit_ids)
    
    h = torch.index_select(messages, 0, index) 
    h = h.view(out_hit_ids.shape + (message_size,))
    h = (h * mask.unsqueeze(2)).sum(1) / (mask.unsqueeze(2).sum(1) + 0.00001)
    
    state = main_net.update_net(h, node_featues)
    return state

In [26]:
main_net = MainNet().to(device)

optimizer = optim.Adam(main_net.parameters(), lr=0.001)

In [ ]:
best_eval = 1000000

in_hit_ids = torch.tensor(in_pmt_ids_val).to(device)
edge_features = torch.tensor(edge_features_val, dtype=torch.float32).to(device)
out_hit_ids = torch.tensor(out_pmt_ids_val).to(device)
mask = torch.tensor(mask_val, dtype=torch.float32).to(device)

for epoch in range(20):
    E_losses = []    
    E_val_losses = []    
    
    for event_id in train.event.unique():
        X_val, E_val, R_val = data_map[event_id]
        node_featues = torch.tensor(X_val, dtype=torch.float32).to(device)
        E = torch.tensor(E_val, dtype=torch.float32).to(device)
        R = torch.tensor(R_val, dtype=torch.float32).to(device)
                    
        optimizer.zero_grad()   
        
        state = main_net.initial_net(node_featues)
        
        state1 = update_state(state)
    
        #e, _ = state1.max(0)
        e = state1.mean(0)
        
        e = e.unsqueeze(0)
    
        E_pred = main_net.predict_net(e)         
    
        loss = ((E_pred - E) ** 2).mean()           
        
        if event_id < 15000:        
            loss.backward()        
            optimizer.step()            
            E_losses.append(loss.item() * E_std ** 2)                
        else:            
            E_val_losses.append(loss.item() * E_std ** 2)            
     
    print(np.mean(E_losses))
    mean_loss = np.mean(E_val_losses)
    print(mean_loss)
    if best_eval > mean_loss:
        best_eval = mean_loss
        torch.save(main_net.state_dict(), "best_E_model.pt")

    #print(np.mean(E_losses), np.mean(R_losses))
    #print(np.mean(E_val_losses), np.mean(R_val_losses))
    #print(np.mean(E_val_losses) * 100 + np.mean(R_val_losses) / 100000)     

1.6520596517198738
0.4855975015458321
0.15371270721327857
0.12993779725287008
0.11228938610230728
0.10669194611173491
0.10252321367907158
0.09814336514771563
0.0973552385445314
0.09609802640735476
0.09378571673489901
0.08881607710150116
0.09128932013720908
0.08859763970182914


In [ ]:
main_net = MainNet().to(device)
optimizer = optim.Adam(main_net.parameters(), lr=0.001)

In [ ]:
best_eval = 1000000

in_hit_ids = torch.tensor(in_pmt_ids_val).to(device)
edge_features = torch.tensor(edge_features_val, dtype=torch.float32).to(device)
out_hit_ids = torch.tensor(out_pmt_ids_val).to(device)
mask = torch.tensor(mask_val, dtype=torch.float32).to(device)

for epoch in range(20):
    R_losses = []    
    R_val_losses = []    
    
    for event_id in train.event.unique():
        X_val, E_val, R_val = data_map[event_id]
        node_featues = torch.tensor(X_val, dtype=torch.float32).to(device)
        E = torch.tensor(E_val, dtype=torch.float32).to(device)
        R = torch.tensor(R_val, dtype=torch.float32).to(device)                

        optimizer.zero_grad()   
        
        state = main_net.initial_net(node_featues)
        
        state1 = update_state(state)
    
        #e, _ = state1.max(0)
        e = state1.mean(0)
        
        e = e.unsqueeze(0)
    
        R_pred = main_net.predict_net(e)         
    
        loss = ((R_pred - R) ** 2).mean()           
        
        if event_id < 15000:        
            loss.backward()        
            optimizer.step()            
            R_losses.append(loss.item() * R_std ** 2)                
        else:            
            R_val_losses.append(loss.item() * R_std ** 2)            
     
    print(np.mean(E_losses))
    mean_loss = np.mean(E_val_losses)
    print(mean_loss)
    if best_eval > mean_loss:
        best_eval = mean_loss
        torch.save(main_net.state_dict(), "best_R_model.pt")